In [9]:
import pandas as pd
import seaborn as sns
from urllib.request import urlretrieve
import re

table_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/metadata.parquet'
urlretrieve(table_url, 'metadata.parquet')

# Read the table using Pandas
metadata_df = pd.read_parquet('metadata.parquet')

Matplotlib is building the font cache; this may take a moment.


In [10]:
metadata_df

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
0,2217ccbd-a1c6-47ac-9a2d-79649727c834.png,"a portrait of a female robot made from code, v...",1,2615452544,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:56:00+00:00,0.554853,0.001621
1,c78807b7-d55a-4a2d-a6b6-9192b18941ad.png,a portrait of a female robot made from a cloud...,1,4231506957,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:59:00+00:00,0.153645,0.000707
2,dc71658a-5e4b-4dca-861a-e1535510348b.png,"only memories remain, trending on artstation",1,544607824,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 12:02:00+00:00,0.062496,0.000425
3,48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png,dream swimming pool with nobody,1,3500509234,50,7.0,8,512,512,019084defdd44d2dfcd200bf7aaae53034263168e10d6b...,2022-08-20 07:22:00+00:00,0.030799,0.003586
4,601d9792-eccd-4850-97a7-edbe91d3464c.png,a dog doing weights. epic oil painting.,1,1312381211,50,12.0,8,512,768,03cb377ebf364ea2761b5710625974c0d2d25285fec982...,2022-08-20 12:28:00+00:00,0.181035,0.030822
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,4a34854e-e117-4b69-b1da-cc74534524c0.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:51:00+00:00,0.152736,0.022676
1999996,d4ae6d4f-762c-42f0-9481-a1c233e6a541.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:53:00+00:00,0.127884,0.014115
1999997,67dac228-3631-4036-8498-a40a67a7d354.png,funny computer,2000,3602197422,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 05:41:00+00:00,0.093476,0.000463
1999998,9d57179b-de12-460b-a699-a66dfca528be.png,hilarious witty computing machine,2000,1604720580,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 06:47:00+00:00,0.124379,0.002547


In [11]:
T1Terms = []
T2Terms = []
T3Terms = []
T4Terms = []

In [12]:

# Our harm dictionary
# Organized as so: [HarmScore, Violent (0-1), Explicit (0-1)] 
harm_dict = {
    "porn": [10, 0 ,1],
    "nude": [8, 0, 1],
    "erotic": [6, 0, 1],
    "bondage": [8, 0, 1],
    "boobs": [5, 0, 1],
    "creampie": [6, 0, 1],
    "fisting": [8, 1, 1],
    "gangbang": [7, 1, 1],
    "gore": [8, 1, 0],
    "hentai": [10, 0, 1],
    "lolicon": [10, 0, 1],
    "lolita": [10, 0, 1],  # in explicit sexual context
    "moaning": [5, 0, 1],
    "nsfw": [8, 0, 1],
    "sex": [10, 0, 1],
    "sexual": [8, 0, 1],
    "shota": [10, 0, 1],
    "teen": [4, 0, 0],  # teen references in sexual context
    "tits": [6, 0, 1],
    "woman": [2, 0, 0],
    "girl": [4, 0, 0],
}


def get_prompt_harm_score(prompt, harm_map):
    tokens = prompt.lower().split()
    total_score = 0
    for token in tokens:
        # If the token exists, get its list, then take the 0th index as the harm score
        score_list = harm_map.get(token, [0, 0, 0])  # default to [0,0,0] if not found
        harm_score = score_list[0]  # assume 0th index is your numeric score
        total_score += harm_score
    return total_score


# Calculate harm scores for each prompt
metadata_df['harm_score'] = metadata_df['prompt'].apply(
    lambda x: get_prompt_harm_score(x, harm_dict)
)

print("Finished")


Finished


In [ ]:
metadata_df['harm_score'].max()

np.int64(40)

In [14]:
print(metadata_df)

                                       image_name  \
0        2217ccbd-a1c6-47ac-9a2d-79649727c834.png   
1        c78807b7-d55a-4a2d-a6b6-9192b18941ad.png   
2        dc71658a-5e4b-4dca-861a-e1535510348b.png   
3        48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png   
4        601d9792-eccd-4850-97a7-edbe91d3464c.png   
...                                           ...   
1999995  4a34854e-e117-4b69-b1da-cc74534524c0.png   
1999996  d4ae6d4f-762c-42f0-9481-a1c233e6a541.png   
1999997  67dac228-3631-4036-8498-a40a67a7d354.png   
1999998  9d57179b-de12-460b-a699-a66dfca528be.png   
1999999  0ae508b1-f140-48c9-9da4-a124967215cf.png   

                                                    prompt  part_id  \
0        a portrait of a female robot made from code, v...        1   
1        a portrait of a female robot made from a cloud...        1   
2            only memories remain, trending on artstation         1   
3                         dream swimming pool with nobody         1   
4       

In [15]:
filteredData = metadata_df.sort_values(by="harm_score", ascending=False)

In [16]:
filteredData

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,harm_score
172230,d82b8a74-370e-416d-b192-0aa8463d2c56.png,A mechanical-steampunk themed lolita outfit ; ...,173,3491905876,50,7.0,8,512,512,373c26cf546e435d5025be53574419c4322c509555e750...,2022-08-09 05:39:00+00:00,0.148281,0.000387,40
1404788,f5dbea23-0133-4861-95ea-43144bdce0ce.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,2265294034,50,11.0,8,512,768,deleted_acount,2022-08-13 11:39:00+00:00,0.801473,0.085923,26
1404789,ca58035b-d8c1-460f-8003-46151117e628.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,3324874849,50,11.0,8,512,768,deleted_acount,2022-08-13 11:57:00+00:00,0.793580,0.276294,26
1404335,dd90c4b9-1d42-48cf-b005-f884f5d21055.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,1205710592,50,11.0,8,512,768,3b93ce8c088a109c99655d40996eb8168ade12fcd93516...,2022-08-13 11:48:00+00:00,0.900430,0.102768,26
1404336,2acae79e-cb19-4750-8405-3c4f4de8ae9f.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,1247897719,50,11.0,8,512,768,3b93ce8c088a109c99655d40996eb8168ade12fcd93516...,2022-08-13 12:00:00+00:00,0.146162,0.131038,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,bcbb05c8-692e-4c33-8793-a9e31db05b40.png,a painting by edward hopper of an eerie scene ...,1,3865331509,50,6.0,8,512,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 07:00:00+00:00,0.089661,0.003524,0
75688,ca284146-268c-4397-a122-997d0a51696d.png,orientalism portrait of a bearded varangian gu...,76,444252399,50,7.0,8,512,512,b49818ac04f3fa4ff6ffe4892fdf4a489f8ef09b633e39...,2022-08-12 00:43:00+00:00,0.087305,0.000218,0
29,09c17453-0080-4838-81b3-dab68fd6eb7d.png,a painting by edward hopper of a group of four...,1,3599045115,50,6.0,8,512,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 07:03:00+00:00,0.101960,0.033366,0
30,7253bec2-7b34-4485-a915-0bc4a0a222ba.png,a painting by edward hopper of a man holding h...,1,1958616805,50,6.0,8,512,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 07:05:00+00:00,0.081142,0.004624,0


In [17]:
metadata_df['prompt_lower'] = metadata_df['prompt'].str.lower()

terms_found = []
for term in harm_dict.keys():
    # Use a word-boundary regex to avoid partial matches
    pattern = rf"\b{re.escape(term)}\b"
    
    # Check if at least one row in 'prompt_lower' contains this term
    if metadata_df['prompt_lower'].str.contains(pattern, regex=True).any():
        terms_found.append(term)

print("Terms found in the dataset:")
print(terms_found)

Terms found in the dataset:
['porn', 'nude', 'erotic', 'bondage', 'boobs', 'creampie', 'fisting', 'gangbang', 'gore', 'hentai', 'lolita', 'moaning', 'nsfw', 'sex', 'sexual', 'shota', 'teen', 'tits', 'woman', 'girl']


In [18]:
## Muitiplication Factor(Term) / Total (Term[s]) 

In [19]:
##Scalability depending on the term T1, T2, T3, T4 Terms. If T1 term is combined with T2 Term = T4. Ex. "Lolita" & "T2" = Likely CP -> T4